<a href="https://colab.research.google.com/github/wai-ming-chan/fed_avg/blob/main/%5BQua%5D_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Communication-Efficient Learning of Deep Networks from Decentralized Data

# Benchmark - Federated Average Learning [McMahan17]


## Helper files

In [1]:
# import global dependencies
import matplotlib
import matplotlib.pyplot as plt
import copy
import numpy as np
from torchvision import datasets, transforms
import torch
import random
from torch import nn

In [2]:
from torch import autograd
from torch.utils.data import DataLoader, Dataset
from sklearn import metrics


class DatasetSplit(Dataset):
    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = list(idxs)

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        image, label = self.dataset[self.idxs[item]]
        return image, label


class LocalUpdate(object):
    def __init__(self, args, dataset=None, idxs=None):
        self.args = args
        self.loss_func = nn.CrossEntropyLoss()
        self.selected_clients = []
        self.ldr_train = DataLoader(DatasetSplit(dataset, idxs), batch_size=self.args.local_bs, shuffle=True)

    def train(self, net):
        net.train()
        # train and update
        optimizer = torch.optim.SGD(net.parameters(), lr=self.args.lr, momentum=0.5)

        epoch_loss = []
        for iter in range(self.args.local_ep):
            batch_loss = []
            for batch_idx, (images, labels) in enumerate(self.ldr_train):
                images, labels = images.to(self.args.device), labels.to(self.args.device)
                net.zero_grad()
                log_probs = net(images)
                loss = self.loss_func(log_probs, labels)
                loss.backward()
                optimizer.step()
                if self.args.verbose and batch_idx % 10 == 0:
                    print('Update Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        iter, batch_idx * len(images), len(self.ldr_train.dataset),
                               100. * batch_idx / len(self.ldr_train), loss.item()))
                batch_loss.append(loss.item())
            epoch_loss.append(sum(batch_loss)/len(batch_loss))
        return net.state_dict(), sum(epoch_loss) / len(epoch_loss)

In [3]:
class CNNMnist(nn.Module):
    def __init__(self, args):
        super(CNNMnist, self).__init__()
        self.conv1 = nn.Conv2d(args.num_channels, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, args.num_classes)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, x.shape[1]*x.shape[2]*x.shape[3])
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

Server function to calculate the average of model parameters received from the current sets of clients


In [4]:
import copy

# Server function to calculate the average of model parameters received from the current sets of clients
def FedAvg(w, clients):
    w_avg = copy.deepcopy(w[0])
    for k in w_avg.keys():
        for i in range(1, len(w)):
            tens = torch.mul(w[i][k], clients[i])
            w_avg[k] += tens
        w_avg[k] = torch.div(w_avg[k], sum(clients))
    return w_avg

In [5]:
import torch.nn.functional as F
from torch.utils.data import DataLoader


def test_img(net_g, datatest, args):
    net_g.eval()
    # testing
    test_loss = 0
    correct = 0
    data_loader = DataLoader(datatest, batch_size=args.bs)
    l = len(data_loader)
    for idx, (data, target) in enumerate(data_loader):
        if args.gpu != -1:
            data, target = data.cuda(), target.cuda()
        log_probs = net_g(data)
        # sum up batch loss
        test_loss += F.cross_entropy(log_probs, target, reduction='sum').item()
        # get the index of the max log-probability
        y_pred = log_probs.data.max(1, keepdim=True)[1]
        correct += y_pred.eq(target.data.view_as(y_pred)).long().cpu().sum()

    test_loss /= len(data_loader.dataset)
    accuracy = 100.00 * correct / len(data_loader.dataset)
    if args.verbose:
        print('\nTest set: Average loss: {:.4f} \nAccuracy: {}/{} ({:.2f}%)\n'.format(
            test_loss, correct, len(data_loader.dataset), accuracy))
    return accuracy, test_loss


In [6]:
# Function to separate training data to users in an IID manner
def mnist_iid(dataset, num_users):
    """
    Sample I.I.D. client data from MNIST dataset
    :param dataset:
    :param num_users:
    :return: dict of image index
    """
    num_items = int(len(dataset)/num_users)
    dict_users, all_idxs = {}, [i for i in range(len(dataset))]
    for i in range(num_users):
        dict_users[i] = set(np.random.choice(
            all_idxs,
            random.randint(1,num_items),
            replace=False))
        print(len(dict_users[i]))
        all_idxs = list(set(all_idxs) - dict_users[i])
    return dict_users

## Data in Uniform Distribution







In [7]:
# parse args
class args:
    gpu = -1 # <- -1 if no GPU is available
    device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')
    num_channels = 1
    num_users = 100
    num_classes = 10
    frac = 0.1
    lr = 0.1
    verbose = 0
    bs = 128
    epochs = 100
    
    iid = True        # < -This Value needs to be changed
    local_ep = 20     # <- This Value needs to be changed
    local_bs = 10     # <- This Value needs to be changed

trans_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
dataset_train = datasets.MNIST('../data/mnist/', train=True, download=True, transform=trans_mnist)
dataset_test = datasets.MNIST('../data/mnist/', train=False, download=True, transform=trans_mnist)
# sample users
if args.iid:
    dict_users = mnist_iid(dataset_train, args.num_users)
else:
    dict_users = mnist_noniid(dataset_train, args.num_users)

img_size = dataset_train[0][0].shape

# build model

net_glob = CNNMnist(args=args).to(args.device)
print(net_glob)
net_glob.train()

# copy weights
w_glob = net_glob.state_dict()

# training
loss_train = []
cv_loss, cv_acc = [], []
val_loss_pre, counter = 0, 0
net_best = None
best_loss = None
val_acc_list, net_list = [], []

199
295
569
494
140
335
568
372
356
499
277
560
127
1
344
187
266
531
103
268
37
587
168
197
285
406
169
119
390
152
472
395
42
135
394
586
351
383
396
408
388
548
291
541
574
587
574
433
120
119
479
55
137
76
582
46
129
422
56
9
212
194
181
380
324
215
258
108
538
441
87
63
515
599
63
71
460
593
8
260
24
110
400
405
421
4
435
22
597
46
168
359
245
25
236
288
501
340
490
151
CNNMnist(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


In [ ]:
for iter in range(args.epochs):

    # w_locals, loss_locals = [], []
    w_locals, loss_locals, num_items = [], [], []
    m = max(int(args.frac * args.num_users), 1)
    idxs_users = np.random.choice(range(args.num_users), m, replace=False)
    for idx in idxs_users:
      num_items.append(len(dict_users[idx]))  
      local = LocalUpdate(args=args, dataset=dataset_train, idxs=dict_users[idx])
      w, loss = local.train(net=copy.deepcopy(net_glob).to(args.device))
      w_locals.append(copy.deepcopy(w))
      loss_locals.append(copy.deepcopy(loss))
    # update global weights
    w_glob = FedAvg(w_locals, num_items)

    # copy weight to net_glob
    net_glob.load_state_dict(w_glob)

    # print loss
    loss_avg = sum(loss_locals) / len(loss_locals)
    
    loss_train.append(loss_avg)
    
    # Evaluate score
    net_glob.eval()
    acc_test, loss_test = test_img(net_glob, dataset_test, args)
    print('Round {:3d}, Average loss {:.3f}, Accuracy {:.3f}'.format(iter, loss_avg, acc_test))

# testing
net_glob.eval()
acc_train, loss_train = test_img(net_glob, dataset_train, args)
acc_test, loss_test = test_img(net_glob, dataset_test, args)
print("Training accuracy: {:.2f}".format(acc_train))
print("Testing accuracy: {:.2f}".format(acc_test))

Round   0, Average loss 1.841, Accuracy 11.350
Round   1, Average loss 1.205, Accuracy 57.870


## Uneven Distribution

In [ ]:
for iter in range(args.epochs):

    # w_locals, loss_locals = [], []
    w_locals, loss_locals, num_items = [], [], []
    m = max(int(args.frac * args.num_users), 1)
    idxs_users = np.random.choice(range(args.num_users), m, replace=False)
    for idx in idxs_users:
      num_items.append(len(dict_users[idx]))
      local = LocalUpdate(args=args, dataset=dataset_train, idxs=dict_users[idx])
      w, loss = local.train(net=copy.deepcopy(net_glob).to(args.device))
      w_locals.append(copy.deepcopy(w))
      loss_locals.append(copy.deepcopy(loss))
    # update global weights
    w_glob = FedAvg(w_locals, num_items)

    # copy weight to net_glob
    net_glob.load_state_dict(w_glob)

    # print loss
    loss_avg = sum(loss_locals) / len(loss_locals)
    
    loss_train.append(loss_avg)
    
    # Evaluate score
    net_glob.eval()
    acc_test, loss_test = test_img(net_glob, dataset_test, args)
    print('Round {:3d}, Average loss {:.3f}, Accuracy {:.3f}'.format(iter, loss_avg, acc_test))

# testing
net_glob.eval()
acc_train, loss_train = test_img(net_glob, dataset_train, args)
acc_test, loss_test = test_img(net_glob, dataset_test, args)
print("Training accuracy: {:.2f}".format(acc_train))
print("Testing accuracy: {:.2f}".format(acc_test))

## Uneven Distribution with Weights

In [ ]:
for iter in range(args.epochs):

    w_locals, loss_locals, num_items = [], [], []
    m = max(int(args.frac * args.num_users), 1)
    idxs_users = np.random.choice(range(args.num_users), m, replace=False)
    for idx in idxs_users:
        num_items.append(len(dict_users[idx]))
        local = LocalUpdate(args=args, dataset=dataset_train, idxs=dict_users[idx])
        w, loss = local.train(net=copy.deepcopy(net_glob).to(args.device))
        w_locals.append(copy.deepcopy(w))
        loss_locals.append(copy.deepcopy(loss))
    # update global weights
    w_glob = FedAvg(w_locals, num_items)

    # copy weight to net_glob
    net_glob.load_state_dict(w_glob)

    # print loss
    loss_avg = sum(loss_locals) / len(loss_locals)
    
    loss_train.append(loss_avg)
    
    # Evaluate score
    net_glob.eval()
    acc_test, loss_test = test_img(net_glob, dataset_test, args)
    print('Round {:3d}, Average loss {:.3f}, Accuracy {:.3f}'.format(iter, loss_avg, acc_test))

# testing
net_glob.eval()
acc_train, loss_train = test_img(net_glob, dataset_train, args)
acc_test, loss_test = test_img(net_glob, dataset_test, args)
print("Training accuracy: {:.2f}".format(acc_train))
print("Testing accuracy: {:.2f}".format(acc_test))

## Adding Noise

In [ ]:
for iter in range(args.epochs):

    w_locals, loss_locals = [], []
    m = max(int(args.frac * args.num_users), 1)
    idxs_users = np.random.choice(range(args.num_users), m, replace=False)
    for idx in idxs_users:
        local = LocalUpdate(args=args, dataset=dataset_train, idxs=dict_users[idx])
        w, loss = local.train(net=copy.deepcopy(net_glob).to(args.device))
        
        # Add noise to layers
        sigma_squared = 0.3
        for layer in w:
            x = np.random.normal(0,sigma_squared,w[layer].size())
            x = np.reshape(x,w[layer].size())
            x = torch.from_numpy(x)
            w[layer] = w[layer]+x.cuda()

        w_locals.append(copy.deepcopy(w))
        loss_locals.append(copy.deepcopy(loss))
    # update global weights
    w_glob = FedAvg(w_locals)

    # copy weight to net_glob
    net_glob.load_state_dict(w_glob)

    # print loss
    loss_avg = sum(loss_locals) / len(loss_locals)
    
    loss_train.append(loss_avg)
    
    # Evaluate score
    net_glob.eval()
    acc_test, loss_test = test_img(net_glob, dataset_test, args)
    print('Round {:3d}, Average loss {:.3f}, Accuracy {:.3f}'.format(iter, loss_avg, acc_test))

# testing
net_glob.eval()
acc_train, loss_train = test_img(net_glob, dataset_train, args)
acc_test, loss_test = test_img(net_glob, dataset_test, args)
print("Training accuracy: {:.2f}".format(acc_train))
print("Testing accuracy: {:.2f}".format(acc_test))